# Automation for the analyst

A team of analysts prepares the monthly report on the prices of the product selected by the Board. Because they are aware you know Python, they asked you to automate the process. Talking to the team, you have set the following business conditions that enable process automation:

Three report parameters are available:
- **product_group_id**,
- **product**,
- **date**.

Assumptions for each parameter:

1. A parameter may have at most one value,
1. If the parameter is empty we return all records from the group,
1. We assume that the file is always prepared correctly (we want to practice report automation, not error handling).

Based on the above requirements:

1. load the  **config.xlsx** file using `openpyxl`,
1. prepare appropriate conditions to filter data from **product_cleaned.csv**,
1. based on the conditions filter the frame,
1. aggregate the data using a **pivot_table**:
   a) index-product, province,
   b) columns-dates,
   c) value-average product price,
   d) remember to remove 0,
6. save the file to the spreadsheet any way you want.

Hints:

1. You can save individual filtering conditions to variables and then use them all to filter `DataFrame`, the same as writing them all as before i.e. `df.loc[var1 & var2]`
1. If you decide to write with Pandas, be careful with the parameters passed to the function (what happens if you set `index=False`?). Link to the [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html).

In [4]:
import pandas as pd
import openpyxl

wb = openpyxl.load_workbook(r'../../01_Data/config.xlsx')
sheet = wb.active

product_group_id = None
product = None
date = None

In [6]:
for row in sheet.iter_rows(values_only=True):
    if row[0] == "product_group":
        product_group_id = row[1]
    elif row[0] == "product":
        product = row[1]
    elif row[0] == date:
        product = row[1]

In [8]:

df_cleaned = pd.read_csv(
    r'../../01_Data/product_prices_cleaned.csv',
    sep = ";",
    encoding = "UTF-8",
    decimal = ".",
    parse_dates=["date"]
)

In [ ]:
if product_group_id is not None:
    df = df[df["product_group_id"] == product_group_id]

In [ ]:
import pandas as pd
import openpyxl

# 1️⃣ Načtení config.xlsx pomocí openpyxl
config_path = "../../01_Data/config.xlsx"
wb = openpyxl.load_workbook(config_path)
sheet = wb.active  # První (aktivní) list

# 2️⃣ Ruční přiřazení hodnot z jednotlivých řádků
product_group_id = None
product = None
date = None

for row in sheet.iter_rows(values_only=True):
    if row[0] == "product_group":
        product_group_id = row[1]
    elif row[0] == "product":
        product = row[1]
    elif row[0] == "date":
        date = row[1]

# 3️⃣ Načtení datového souboru product_cleaned.csv
df = pd.read_csv("../../01_Data/product_cleaned.csv", sep=";")

# 4️⃣ Filtrace podle podmínek
if product_group_id is not None:
    df = df[df["product_group_id"] == product_group_id]

if product is not None:
    df = df[df["product"] == product]

if date is not None:
    df = df[df["date"] == date]

# 5️⃣ Vytvoření pivot tabulky
pivot = pd.pivot_table(
    data=df,
    index=["product", "province"],
    columns=["date"],
    values=["value"],
    aggfunc="mean"
)

# 6️⃣ Odstranění nulových hodnot
pivot = pivot.replace(0, pd.NA).dropna(how="all")

# 7️⃣ Uložení výsledku do report.xlsx
output_path = "../../01_Data/report.xlsx"
pivot.to_excel(output_path)

print(f"Report byl uložen jako {output_path}")
